In [ ]:
rdd = sc.textFile('/tmp/realEstate.csv')
rdd = rdd.map(lambda line: line.split(","))
header = rdd.first()
rdd = rdd.filter(lambda line:line != header)
df = rdd.map(lambda line: Row(street = line[0], city = line[1], zip=line[2], beds=line[4], baths=line[5], sqft=line[6], price=line[9])).toDF()
df.show(5)

In [ ]:
uberfile = "/tmp/uber-raw-data-noheader.csv"
uberrdd = sc.textFile(uberfile)
def project(rec):
    fields = rec.split(',')
    return fields
    
projection = uberrdd.map(lambda rec: project(rec))
trans = projection.map(lambda l: [ l[2], l[1]])


In [ ]:
import numpy as np
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
# adding the PySpark modul to SparkContext
sc.addPyFile("https://raw.githubusercontent.com/seahboonsiew/pyspark-csv/master/pyspark_csv.py")
import pyspark_csv as pycsv
collisions = sc.textFile("/tmp/NYPD_Motor_Vehicle_Collisions.csv")

collisions_header = collisions.first()
collisions_header_list = collisions_header.split(",")
collisions_body = collisions.filter(lambda line:line != collisions_header)
collisions_body = collisions_body.filter(lambda line : len(line.split(","))>29)
collisions_df = pycsv.csvToDataFrame(sqlContext, collisions_body, sep=",", columns=collisions_header_list)

In [ ]:
import numpy as np
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
# adding the PySpark modul to SparkContext
sc.addPyFile("https://raw.githubusercontent.com/seahboonsiew/pyspark-csv/master/pyspark_csv.py")
import pyspark_csv as pycsv
collisions = sc.textFile("/tmp/NYPD_Motor_Vehicle_Collisions.csv")
def skip_header(idx, iterator):
    if (idx == 0):
        next(iterator)
    return iterator

collisions_header = collisions.first()
collisions_header_list = collisions_header.split(",")
collisions_body = collisions.mapPartitionsWithIndex(skip_header)
collisions_body = collisions_body.filter(lambda line : len(line.split(","))>29)
collisions_df = pycsv.csvToDataFrame(sqlContext, collisions_body, sep=",", columns=collisions_header_list)

In [1]:
import numpy as np

data_file = "/tmp/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
def parse_line(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)

In [ ]:
import numpy as np

data_file = "/tmp/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
def parse_line_with_key(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return (line_split[41], np.array([float(x) for x in clean_line_split]))

label_vector_data = raw_data.map(parse_interaction_with_key)


In [ ]:
def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key).filter(lambda x: x[0]==label)
    return Statistics.colStats(label_vector_data.values())

normal_sum = summary_by_label(raw_data, "normal.")
